In [ ]:
import numpy as np


In [ ]:
import enum
from collections import Counter, defaultdict

class TextContext(enum.IntEnum):
    Standard = 0
    Code = enum.auto()
    Spoiler = enum.auto()
    Url = enum.auto()

class Start: pass
class End: pass

START = Start()
END = End()

Token = str | Start | End
Counts = defaultdict[int, Counter[int]]

tokens: list[Token] = [START, END]
token_ids: dict[Token, int] = {START: 0, END: 1}
token_counts: Counts = defaultdict(Counter)

token_counts[1].update([1])




In [ ]:
def add_token(tok: Token):
    if tok in token_ids:
        return
    id = len(tokens)
    tokens.append(tok)
    token_ids[tok] = id

In [ ]:
import itertools
def add_sequence(toks: list[Token]):
    for tok in toks:
        add_token(tok)
    ids = [token_ids[tok] for tok in toks]
    for x, y in itertools.pairwise(ids):
        token_counts[x].update([y])
    


In [ ]:
def add_sentence(s: str):
    if s:
        global token_counts
        add_sequence([START] + s.split() + [END])

In [ ]:
with open("example.txt") as f:
    add_sentence(f.read())

In [ ]:
def cum_weights():
    return [np.cumsum(list(counter.values())) for counter in token_counts.values()]


In [ ]:
def get_random_id(c: list[np.ndarray], token_id: int) -> int:
    idx = np.searchsorted(c[token_id], np.random.randint(c[token_id][-1]))
    return list(token_counts[token_id].keys())[idx]

In [ ]:
def generate(start: Token | None = None, limit: int = 10):
    c = cum_weights()
    if start is None:
        start = tokens[get_random_id(c, token_ids[START])]

    out: list[Token] = [start]
    current = token_ids[start]

    print(out)
    
    for _ in range(limit):
        current = get_random_id(c, current)
        out.append(tokens[current])
        if tokens[current] is END:
            break
    
    return out

In [ ]:
generate("Why")